In [1]:
import numpy as np
import pandas as pd 
from astroquery.jplhorizons import Horizons
from astropy import constants

Partiendo de la posición y velocidad de Júpiter el día, mes y año de su cumpleaños a la media noche, calcular, usando el método de Euler y el método de Verlet, la posición y velocidad de Júpiter 10 día después. Compare el resultado con el valor obtenido con Astroquery para ese mismo día.  No cambie las unidades de au y días a metros y segundos y para el parámetro gravitacional del Sol use el valor 0.0002959122 au³/d²

In [2]:
#Importando las librerias necesarias
import numpy as np
import pandas as pd 
from astroquery.jplhorizons import Horizons
#from astropy import constants

#Teniendo en cuenta que mi fecha de nacimiento es 2002-07-26, entonces extraigamos las condiciones inciales para Jupiter este día:
Horizons(id='599', location='0', epochs={'start': '2002-01-01', 'stop': '2022-12-31', 'step': '90d'}).vectors().to_pandas()[['datetime_str', 'x', 'y', 'z', 'vx', 'vy', 'vz']]